####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::343278755431:role/GlueRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: df6870ec-dd66-44de-96b7-d1db8b993713
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session df6870ec-dd66-44de-96b

In [ ]:
hits_df = spark.read.format("csv").option("delimiter","\t").option("header", True).load("s3://hit-details/data1.tsv")

def find_top_ref(df, hit_time_gmt, user_agent, ip, page_url):
    # Find the topmost record by recursively finding the record with a NULL referrer
    topmost = df.filter((col("hit_time_gmt") <= hit_time_gmt) & (col("user_agent") == user_agent) & (col("ip") == ip) & (col("page_url") == page_url)).select("hit_time_gmt", "user_agent", "ip", "page_url", "referrer")
    referrer = df.filter((col("hit_time_gmt") <= hit_time_gmt) & (col("user_agent") == user_agent) & (col("ip") == ip) & (col("page_url") == page_url)).select("referrer").collect()[0][0]
    
    if 'esshopzilla' not in referrer:
        print("False")
    if referrer is not None:
        if 'esshopzilla' not in referrer:
            return topmost
        else:
            topmost = find_top(df,hit_time_gmt, user_agent, ip, referrer)
    
    # Return the topmost record
    return topmost
    
    

#Call the recursive function to find the topmost and bottommost records for each parent-child relationship
result = hits_df.filter("pagename=='Order Complete'").select("hit_time_gmt", "user_agent", "ip", "page_url", "referrer", "product_list",   "pagename").rdd.flatMap(lambda row: find_top(hits_df, row[0], row[1], row[2], row[3])).toDF()
result.show()

